In [1]:
import nltk
import string
import re
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist


def preprocess_comment(comments):

    remove_punc = str.maketrans('', '', string.punctuation)
    Comments_without_punc = comments.translate(remove_punc)
    
    comments_without_whitespaces = " ".join(Comments_without_punc.split())
   
    special_char = r'[^a-zA-Z0-9\s]' 
    comments_without_special_char = re.sub(special_char, '', comments_without_whitespaces)
   
    comments_final = "< " + comments_without_special_char + " >"
    
    tokens = word_tokenize(comments_final.lower())
    
    return tokens

def smoothing(t,s,v):
    
    count = (t+1) / (s+v)
    return count
    
def perplexity(probability,word_count):
    #find
    perp = pow(probability, (-1 / word_count))
    return perp

def probability(uni_Freq, bigram_Freq, comment):
    #print(comment)
    probability = 1.0
    multy_prob = 0.0
    for i in range(len(comment)):
        
        if comment[i] in bigram_Freq:
            #print(comment[i])
            t = bigram_Freq[comment[i]]
            #print(t)
        else:
            t = 0
        if comment[i][0] in uni_Freq:
            #print(comment[i][0])
            s = uni_Freq[comment[i][0]]
            #print(s)
        else:
            s = 0
        #print(t)
        #print(s)
        probability = probability * smoothing(t,s,len(uni_Freq))
        multy_prob = multy_prob + probability
    return multy_prob
    
#train data sets
def train_comments(comment,a):
    
    uni_list = []
    bigram_list = []
   
    #preprocessing
    for line in a:
        
        tokens = preprocess_comment(line)
        #unigrams
        uni_list = uni_list + list(tokens)
        #bigrams  
        bigram_comments = nltk.bigrams(tokens)
        bigram_list = bigram_list + list(bigram_comments)
    
    uni_Freq = FreqDist(uni_list)
    bigram_Freq = FreqDist(bigram_list)
    probability_of_comment = probability(uni_Freq, bigram_Freq, comment)
    
    return probability_of_comment
        

    
#open negative corpus
with open ('negative.txt', 'rt',encoding='utf-8') as text_file:  
    extracted_negative_text = text_file.readlines() 
#text_file.close()

#open positive corpus
with open ('positive.txt', 'rt',encoding='utf-8') as text_file:  
    extracted_positive_text = text_file.readlines() 
#text_file.close()
        
#testing
print("Enter the comment : ")
new_comment = input()
new_comment_tokens = preprocess_comment(new_comment)
new_comment_bigrams = list(nltk.bigrams(new_comment_tokens))
word_count = len(new_comment_tokens)

positive_prob = train_comments(new_comment_bigrams, extracted_positive_text)
negative_prob = train_comments(new_comment_bigrams, extracted_negative_text)

print("positive probability of the comments is : "+str(positive_prob))
print("negative probability of the comments is : "+str(negative_prob))

if positive_prob > negative_prob:
    
    perp = perplexity(positive_prob,word_count)
    print("This comment is a positive comment and perplexity is : " +str(perp))

else:
    if negative_prob > positive_prob :
        perp = perplexity(negative_prob,word_count)
        print("This comment is a negative comment and perplexity is : " +str(perp))
    
    else :
        print("This comment is neutral")


        

Enter the comment : 
mahinda hora
positive probability of the comments is : 0.00898527564653457
negative probability of the comments is : 0.0026746506274597163
This comment is a positive comment and perplexity is : 3.2480084379056002
